# SVD – Décomposition en valeurs singulières

### Théorie mathématique


La théorie mathématique dit que :
    
Soit M $\in$ $\mathbb{R}^{n \times p}$. On peut trouver :

   - U $\in$ $\mathbb{R}^{n \times n}$ et V $\in$ $\mathbb{R}^{p \times p}$, orthogonales (UU$^\intercal$ = 0, VV$^\intercal$ = 0)
   - Une matrice 'diagonale' $\Sigma$ $\in$ $\mathbb{R}^{n \times p}$ de coefficients diagonaux positifs ou nuls, 
telle que 

\begin{equation}
   M = U \Sigma V^\intercal
\end{equation}

Cette décomposition est la SVD de M.

On a alors : 


\begin{equation}
\Sigma = 
\begin{pmatrix}
\begin{array}{c c c | c}
\sigma_1  &    0    &    0   & \\
    0     & \ddots  &    0   &  0\\ 
    0     &  0    & \sigma_n & \\
 \end{array}
\end{pmatrix}
\end{equation}


\begin{equation}
\Sigma = 
\begin{pmatrix}
\begin{array}{c c c}
\sigma_1  &    0    &    0    \\
    0     & \ddots  &    0    \\ 
    0     &    0    & \sigma_p  \\ \hline 
          &    0    &
 \end{array}
\end{pmatrix}
\end{equation}


Voici un exemple simple de SVD avec Numpy.

In [44]:
import numpy as np
from numpy import linalg
M = np.matrix([[1, 2, 3], [4, 5, 6]])
U, s, V = linalg.svd(M)

print('U = {}'.format(np.array(U)))

U = [[-0.3863177  -0.92236578]
 [-0.92236578  0.3863177 ]]


In [45]:
print('s = {}'.format(np.array(s)))

s = [9.508032   0.77286964]


In [46]:
print('V = {}'.format(np.array(V)))

V = [[-0.42866713 -0.56630692 -0.7039467 ]
 [ 0.80596391  0.11238241 -0.58119908]
 [ 0.40824829 -0.81649658  0.40824829]]


In [2]:
import pandas as pd
from surprise import Reader
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
reader = Reader(rating_scale=(1,5))

In [4]:
ratings = pd.read_csv('/Users/alessandro/Desktop/pfe/samples/samples100k/sample0.csv')
ratings.head(5)

,movie_id,customer_id,rating,date
0,15093,294081,5,2004-01-21
1,2743,1965814,4,2005-08-23
2,3624,2143181,4,2004-07-15
3,17295,2218945,4,2005-04-14
4,12355,187135,3,2005-12-01


In [60]:
# Transform the dataframe in a dataset object
data = Dataset.load_from_df(ratings[['customer_id', 'movie_id', 'rating']], reader)

In [48]:
# sample random trainset and testset
# test set is made of 20% of the ratings.
trainset, testset = train_test_split(data, test_size=.20)

L'algo de SVD va permettre de réduire les composantes d'un dataset en reduisant les dimensions de la matrice. De cet algo et des 3 matrices qu'il nous reste, on va pouvoir faire des prédictions.

On a donc : 
\begin{equation}
   M = U \Sigma V^\intercal
\end{equation}


On pose : 

- $U^{n \times r}$ la matrice des valeurs singulières à gauche avec *n* les films et *r* les "features" du film.
- $\Sigma^{r \times r}$ la matrice diagonale des valeurs singulières.
- $V^{n \times r}$ la matrice des valeurs singulières à droite avec *n* les utilsiateurs et *r* les "features" de chaque utilisateur.

L'algorithme de SVD a pour rôle de réduire les dimensions de notre matrice et en faisant cela, il représente les profils des utilisateurs mais aussi les profils des différents films.

La contrainte est que notre matrice est **sparse**, elle est lacunaire. Ainsi la SVD de notre matrice n'existe pas.

Cependant, il est possible de déterminer U et V si l'on arrive à déterminer les vecteurs $p_u$ et $q_i$. Ces vecteurs composent respectivement les lignes de U et les colonnes de V$^\intercal$.

Pour trouver les vecteurs $p_u$ et $q_i$, on doit résoudre ce problème d'optimisation et utiliser le SGD (Stochastic Gradient Descent) :
\begin{equation}
    (\min \sum_{r_{ui} \in R} (r_{ui} - p_u \cdot q_i)^2) \text{ tel que $r_ui = p_u \cdotp q_i$}
\end{equation} 


Le SGD nous permet de trouver plus simplement le minimum d'un fonction.
Dans notre cas, l'équation donne :

\begin{equation}
\sum_{r_{ui} \in R} (r_{ui} - p_u \cdot q_i)^2 =\sum_{r_{ui}
\in R} f_{ui}(p_u, q_i)
\end{equation}


Une fois $p_u$ et $q_i$ calculés, on peut estimer les notes utilisateurs par la formule: 
\begin{equation}
\hat{r}_{ui} = p_u \cdot q_i
\end{equation}


In [49]:
# We'll use the famous SVD algorithm.
algo = SVD()

In [50]:
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)

In [51]:
predictions = algo.test(testset)

On calcul l'erreur quadratique moyenne (RMSE — Root Mean Square Error) qui en quelque sorte est une moyenne des erreurs de prediction.

\begin{equation}
RMSE = \sqrt{\sum_{u,i} (\hat{r}_{ui} - r_{ui})^2}
\end{equation}


In [52]:
# Then compute RMSE (Root Mean Square Error), wich is some sort of average error, where big errors are heavily penalized
accuracy.rmse(predictions)

RMSE: 1.0435


1.0435263512740693